In [27]:
def check_files_existance(files):
    for f in files:
        if not os.path.exists(f):
            print(f"warning could not find file: {f}")
            return False
    return True
    
def process_language_pair_verses(a_file, i_file): 
    ###
    # receives an alignment and an index file and returns verses mapped to their alignments only for eng-mixed and fra-louissegond editions
    # a_file: alignment_file
    # i_file: alignments index file
    ###

    print(f"going to process {a_file}, {i_file}")
    res = {}
    
    if check_files_existance([a_file, i_file]):
        with open(a_file, 'r') as afl, open(i_file, 'r') as ifl:
            a_lines = afl.readlines()
            i_lines = ifl.readlines()

            for tup in zip(a_lines, i_lines):
                aligns = tup[0].split('\t')[1]
                verse, s_edit, t_edit = tuple(tup[1].strip().split())
                if s_edit == 'eng-x-bible-mixed' and t_edit == 'fra-x-bible-louissegond':
                    res[verse] = aligns
    return res


def update_alignments(size, alignments):
    a_file = f'/mounts/work/ayyoob/alignment/output_various_size/eflomal_aligns/eng_fra_word_{size}.gdfa'
    i_file = f'/mounts/work/ayyoob/alignment/output_various_size/index_eng_fra.txt_{size}'
    editf1 = 'eng-x-bible-mixed'
    editf2 = 'fra-x-bible-louissegond'
    measures = {}
    measures['intersectionold'] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
    measures['intersectionnew'] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
    res = process_language_pair_verses(a_file, i_file)
    for i,verse in enumerate(res.keys()):
        if verse in alignments:
            #inter_aligns = autils.get_aligns(editf1, editf2, alignments[verse])
            #autils.calc_and_update_alignment_score(inter_aligns, pros[verse], surs[verse], measures['intersectionold'])
            #prev = alignments[verse]['eng-x-bible-mixed']['fra-x-bible-louissegond']
            #alignments[verse]['eng-x-bible-mixed']['fra-x-bible-louissegond'] = res[verse]
            #inter_aligns = autils.get_aligns(editf1, editf2, alignments[verse])
            #autils.calc_and_update_alignment_score(inter_aligns, pros[verse], surs[verse], measures['intersectionnew'])
            #alignments[verse]['eng-x-bible-mixed']['fra-x-bible-louissegond'] = prev
            
            alignments[verse]['eng-x-bible-mixed']['fra-x-bible-louissegond'] = res[verse]
            
    #print(measures['intersectionold'])
    #print(measures['intersectionnew'])

update_alignments('500000', verse_alignments_inter)

going to process /mounts/work/ayyoob/alignment/output_various_size/eflomal_aligns/eng_fra_word_500000.gdfa, /mounts/work/ayyoob/alignment/output_various_size/index_eng_fra.txt_500000
{'p_hit_count': 4503, 's_hit_count': 4394, 'total_hit_count': 4639, 'gold_s_hit_count': 8428, 'prec': 0.971, 'rec': 0.521, 'f1': 0.678, 'aer': 0.319}
{'p_hit_count': 5964, 's_hit_count': 5617, 'total_hit_count': 6710, 'gold_s_hit_count': 8428, 'prec': 0.889, 'rec': 0.666, 'f1': 0.762, 'aer': 0.235}


In [1]:
import torch, sys
sys.path.insert(0, '../')
from my_utils import gpu_utils
import importlib, gc
from my_utils.alignment_features import *
import my_utils.alignment_features as afeatures
importlib.reload(afeatures)
import gnn_utils.graph_utils as gutils

/mounts/Users/student/ayyoob/anaconda3/envs/multalign_graph/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# !pip install torch-geometric
# !pip install tensorboardX

# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

#  print(torch.version.cuda)
#  print(torch.__version__)    

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
free_gpu1 = '6'
free_gpu2 = '5'


In [9]:

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt




In [2]:
from my_utils import align_utils as autils, utils
import argparse
from multiprocessing import Pool

# set random seed
config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
utils.setup(config_file)

params = argparse.Namespace()
params.gold_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/eng_fra_pbc/eng-fra.gold"
params.editions_file =  "/mounts/Users/student/ayyoob/Dokumente/code/pbc_utils/data/helfi/splits/helfi_lang_list.txt"
params.core_count = 50

pros, surs = autils.load_gold(params.gold_file)
all_verses = list(pros.keys())
editions, langs = autils.load_simalign_editions(params.editions_file)
current_editions = [editions[lang] for lang in langs]

def get_pruned_verse_alignments(args):
    verse, current_editions = args
    verse_aligns_inter = autils.get_verse_alignments(verse)
    verse_aligns_gdfa = autils.get_verse_alignments(verse, gdfa=True)

    autils.prune_non_necessary_alignments(verse_aligns_inter, current_editions)
    autils.prune_non_necessary_alignments(verse_aligns_gdfa, current_editions)

    gc.collect()
    return verse_aligns_inter, verse_aligns_gdfa
    

verse_alignments_inter = {}
verse_alignments_gdfa = {}
args = []
for i,verse in enumerate(all_verses):
    args.append((verse, current_editions[:]))

with Pool(30) as p:
    all_res = p.map(get_pruned_verse_alignments, args)

for i,verse in enumerate(all_verses):
    verse_aligns_inter, verse_aligns_gdfa = all_res[i]
    
    verse_alignments_inter[verse] = verse_aligns_inter
    verse_alignments_gdfa[verse] = verse_aligns_gdfa

for verse in all_verses[:]:
    if len(verse_alignments_inter[verse].keys()) < 2:
        all_verses.remove(verse)
        
utils.LOG.info("done")

2021-09-03 16:22:32,901 - analytics - INFO - done


In [5]:
class DeeperGCN(torch.nn.Module):
    def __init__(self,node_dim, edge_dim, hidden_channels, num_layers, features):
        super(DeeperGCN, self).__init__()

        self.node_encoder = nn.Linear(node_dim, hidden_channels)
        self.edge_encoder = nn.Linear(edge_dim, hidden_channels)

        self.layers = torch.nn.ModuleList()
        for i in range(1, num_layers + 1):
            conv = pyg_nn.GENConv(hidden_channels, hidden_channels, aggr='softmax',
                           t=1.0, learn_t=True, num_layers=2, norm='layer')
            norm = nn.LayerNorm(hidden_channels, elementwise_affine=True)
            act = nn.ReLU(inplace=True)

            layer = pyg_nn.DeepGCNLayer(conv, norm, act, block='res+', dropout=0.1,
                                 ckpt_grad=i % 3)
            self.layers.append(layer)

        self.already_inited = False
        self.prev_edge_index = None
        self.prev_edge_attr = None

        self.feature_encoder = FeatureEncoding(features)
        
    def forward(self, x, edge_index):
        x = self.feature_encoder(x)
        x = self.node_encoder(x)

        if not self.already_inited or self.prev_edge_index.data_ptr() != edge_index.data_ptr():
            edge_index_np = edge_index.cpu().numpy()
            val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
            vals = x_edge_vals[val_indices, :]
            vals = vals.reshape((vals.shape[1], vals.shape[2]))
            self.prev_edge_attr = vals.to(dev)
            self.prev_edge_index = edge_index
            self.already_inited = True
        edge_attr = self.edge_encoder(self.prev_edge_attr)

        x = self.layers[0].conv(x, edge_index, edge_attr)

        for layer in self.layers[1:]:
            x = layer(x, edge_index, edge_attr)

        x = self.layers[0].act(self.layers[0].norm(x))
        # x = F.dropout(x, p=0.1, training=self.training)

        return x

In [6]:
class Discriminator(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(Discriminator, self).__init__()
        self.lin1 = torch.nn.Linear(in_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, hidden_channels)
        self.lin3 = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x


In [7]:
class Encoder2(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder2, self).__init__()
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels)
        self.conv2 = pyg_nn.GATConv(2 * out_channels , out_channels)

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index, ))
        return self.conv2(x, edge_index)

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, features, n_head = 2, edge_feature_dim = 0,):
        super(Encoder, self).__init__()
        #self.lin = nn.Linear(in_channels, out_channels)
        self.conv1 = pyg_nn.GATConv(in_channels, 2*out_channels, heads= n_head)
        self.conv2 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= 1)
        #self.conv3 = pyg_nn.GATConv(2 * n_head *  out_channels , out_channels, heads= n_head)
        self.f_embedding = nn.Linear(in_channels, in_channels)
        

        self.feature_encoder = afeatures.FeatureEncoding(features)
        #self.already_inited = False
        #self.prev_edge_index = None
        #self.prev_edge_attr = None

    def forward(self, x, edge_index):
        x = self.feature_encoder(x, dev)
        x = F.relu(self.f_embedding(x))
        #if not self.already_inited or self.prev_edge_index.data_ptr() != edge_index.data_ptr():
        #    edge_index_np = edge_index.cpu().numpy()
        #    val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
        #    vals = x_edge_vals[val_indices, :]
        #    vals = vals.reshape((vals.shape[1], vals.shape[2]))
        #    self.prev_edge_attr = vals.to(dev)
        #    self.prev_edge_index = edge_index
        #    self.already_inited = True
        #x = self.lin(x)
        x = F.elu(self.conv1(x, edge_index, ))
        #x = self.conv_gin(x, edge_index)
        #x = F.elu(self.conv2(x, edge_index))
        return self.conv2(x, edge_index)#, self.conv3(x, edge_index)
    

def train(epoch):
    global optimizer
    total_loss = 0
    cluster_loss = 0
    model.train()

    #for i in tqdm(range(int(train_pos_edge_index_permed.shape[1]/batch_size)+1)):
    for i,batch_ in enumerate(tqdm(data_loader)):
        for verse in batch_:
            batch = batch_[verse]
            optimizer.zero_grad()
                
            z = model.encode(batch['x'].to(dev), batch['edge_index'].to(dev))
            #z1 = encoder2(z, torch.tensor(batch['intra_sent_edges'], dtype=torch.long).to(dev))
            #z = torch.cat((z,z1), dim=1)
            #for j in range(5):
            #    discriminator_optimizer.zero_grad()
            #    discriminator_loss = model.discriminator_loss(z) / (int(train_pos_edge_index_permed.shape[1]/batch_size)+1)
            #    discriminator_loss.backward()
            #    discriminator_optimizer.step()
            
            pos = torch.tensor(batch['pos'], dtype=torch.long).to(dev)
            neg = torch.tensor(batch['neg'], dtype=torch.long).to(dev)
            #nodes = torch.tensor(list(batch['nodes']), dtype=torch.long).to(dev)

            loss1 = model.recon_loss( z, pos, neg) #TODO try providing better neg edges
            #ortho_loss, mincut_loss, entropy_loss = model.decoder.clustering_loss(z, nodes, batch['adjacency'])
            
            loss =   loss1 * pos.shape[1] #+ ortho_loss + mincut_loss #+ 0.05 * entropy_loss #* pos.shape[1]/train_neg_edge_index.shape[1] #+ model.reg_loss(z)/(int(train_pos_edge_index_permed.shape[1]/batch_size)+1)# + (1 / x.shape[0]) * model.kl_loss()

            loss.backward()
            optimizer.step()
            total_loss += loss.item() 
            cluster_loss += loss1

            
            #if epoch > 0:
            #    if i  % 500 == 499:
            #        alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses, test_nodes_map,
            #                dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)
            #        # decoder.set_objective('sequence_prediction')
            #        # auc, ap = test(edge_index_seq_sent, edge_index_seq_sent_neg, epoch)
            #        # print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
            #        # if epoch > 4:
            #        #     decoder.set_objective('link_prediction')
            #        model.train()
            
            #if (i+1)*batch_size > train_pos_edge_index.shape[1]:
            #    break

            if i % 50 == 0:
                gc.collect()
                with torch.no_grad():
                    torch.cuda.empty_cache()
    
    writer.add_scalar("loss", total_loss, epoch)
    print(f"train loss: {total_loss}")
    print(f"cluster loss: {cluster_loss}")

def test(pos_edge_index, neg_edge_index, epoch):
    model.eval()
    tot_auc = tot_ap = 0
    with torch.no_grad():
        z = model.encode(x_test, torch.cat((train_pos_edge_index, neg_edge_index), dim=1).to(dev))
        
        neg_pos_coeff = neg_edge_index.shape[1]/ pos_edge_index.shape[1]
        for i in (range(int(pos_edge_index.shape[1]/batch_size)+1)):
            auc,ap = model.test(z, pos_edge_index[:, i*batch_size:(i+1)*batch_size].to(dev),
                                neg_edge_index[:, int(i*batch_size*neg_pos_coeff):int((i+1)*batch_size*neg_pos_coeff)].to(dev))

            tot_auc += auc * pos_edge_index[:, i*batch_size:(i+1)*batch_size].shape[1]
            tot_ap += ap *  pos_edge_index[:, i*batch_size:(i+1)*batch_size].shape[1]


    return tot_auc/pos_edge_index.shape[1], tot_ap/pos_edge_index.shape[1]



In [8]:
EPS = 1e-15

def _diag(x):
    eye = torch.eye(x.size(0)).type_as(x)
    out = eye * x.unsqueeze(1).expand(x.size(0), x.size(0))
    return out

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, edge_features, n_cluster=32):
        super(Decoder, self).__init__()
        self.feature_encoder = afeatures.FeatureEncoding(edge_features)
        self.features_size = sum([x.out_dim for x in edge_features])
        self.representataion_size = (input_size - self.features_size)

        self.transfer = nn.Sequential(nn.Linear(input_size, hidden_size*2), nn.ReLU(), nn.Dropout(drop_out),
                            nn.Linear(hidden_size*2, hidden_size), nn.ReLU(), nn.Dropout(drop_out),
                            nn.Linear(hidden_size, 1))

        #self.transfer = nn.Sequential(nn.ELU(), nn.Linear(n_cluster*2, 1), nn.ELU())

        #self.n_cluster = n_cluster                
        #self.cluster = nn.Sequential(nn.Linear(int((input_size - len(edge_features))/2), hidden_size*2), nn.ELU(), nn.Linear(hidden_size*2, 2*n_cluster))
        #self.actual_cluster = nn.Linear(2*n_cluster, n_cluster)
        #self.cos = nn.CosineSimilarity(dim=1)        
        #self.dist = nn.PairwiseDistance()
        #self.gnn_transform = nn.Sequential(nn.Linear(self.representataion_size, hidden_size), nn.ReLU(), nn.Dropout(drop_out))
        self.counter = 0

        self.objective = 'link_prediction'
    def forward(self, z, edge_index, sigmoid = True):
        if self.features_size > 0:
            if self.objective == 'link_prediction':
                edge_index_np = edge_index.cpu().numpy()
                val_indices = x_edge_np[edge_index_np[0, :], edge_index_np[1, :]]
                val_indices = np.squeeze(np.asarray(val_indices))
                vals = x_edge_vals2[val_indices, :]
            elif self.objective == 'sequence_prediction':
                vals = torch.zeros((edge_index.shape[1], self.features_size)).to(dev)


            features = self.feature_encoder(vals.to(dev), dev)
            #features = vals.to(dev)
            h1 = z[edge_index[0, :]]
            h2 = z[edge_index[1, :]]

            self.counter += 1

            #rep = self.gnn_transform(torch.cat((h1, h2), dim=1))
            res = self.transfer(torch.cat((self.cluster(h1), self.cluster(h2), features), dim=1))
            #res = self.transfer(features)
        else:
            h1 = z[edge_index[0, :]]
            h2 = z[edge_index[1, :]]

            
            res = self.transfer(torch.cat((h1, h2), dim=-1))

            #res = self.transfer(torch.cat((self.cluster(h1), self.cluster(h2)), dim=1))
            #res = torch.sum(torch.pow(F.softmax(self.cluster(h1)/1, dim=1) - F.softmax(self.cluster(h2)/1, dim=1), 2), dim=1)
            #res = self.cos(self.cluster(h1), self.cluster(h2))
            #res = - self.dist(self.cluster(h1), self.cluster(h2))
            #print(res)
        res = torch.sigmoid(res) if sigmoid else res
        return res

    def set_objective(self, objective):
        self.objective = objective
        
    def clustering_loss(self, z, nodes, adjacency):
        s = self.actual_cluster(torch.relu(self.cluster(z[nodes])))
        s = torch.softmax(s, dim=-1)
        entropy_loss = (-s * torch.log(s + EPS)).sum(dim=-1).mean()

        ss = torch.matmul(s.transpose(0, 1), s)
        i_s = torch.eye(self.n_cluster).type_as(ss)
        ortho_loss = torch.norm(
            ss / torch.norm(ss, dim=(-1, -2), keepdim=True) -
            i_s / torch.norm(i_s), dim=(-1, -2))
        ortho_loss = torch.mean(ortho_loss)

        adjacency = adjacency.to(dev).float()
        out_adj = torch.matmul(s.transpose(0, 1),torch.sparse.mm(adjacency, s))
        # MinCUT regularization.
        mincut_num = torch.trace(out_adj)
        #d_flat = torch.einsum('ij->i', adjacency) # FIXME since I don't consider the whole adjacency matrix this could be a source of problem
        d_flat = torch.sparse.sum(adjacency, dim=1).to_dense()
        d = _diag(d_flat)
        mincut_den = torch.trace(
            torch.matmul(torch.matmul(s.transpose(0, 1), d), s))
        mincut_loss = -(mincut_num / mincut_den)
        mincut_loss = torch.mean(mincut_loss)

        return ortho_loss, mincut_loss, entropy_loss
    
    def get_alignments(self, z, edge_index):
        h1 = z[edge_index[0, :]]
        h2 = z[edge_index[1, :]]
        
        h1 = torch.softmax(self.cluster(h1), dim=1)
        h2 = torch.softmax(self.cluster(h2), dim=1)

        h1_max = torch.argmax(h1, dim=1)
        h2_max = torch.argmax(h2, dim=1)

        h1_cluster = torch.zeros(*h1.shape)
        h2_cluster = torch.zeros(*h2.shape)

        h1_cluster[range(h1.size(0)), h1_max] = 1
        h2_cluster[range(h2.size(0)), h2_max] = 1
        res = torch.max(h1_cluster * h2_cluster, dim=1).values

        #res = h1 * h2
        #res = torch.sum(res, dim = 1)
        return torch.unsqueeze(res, dim=1)

In [9]:
import pickle



train_verses = all_verses[:]
test_verses = all_verses[:] 
editf1 = "eng-x-bible-mixed"
editf2 = 'fra-x-bible-louissegond'


small_editions = current_editions[:]

if 'jpn-x-bible-newworld' in small_editions:
    small_editions.remove('jpn-x-bible-newworld')
if 'grc-x-bible-unaccented' in small_editions:
    small_editions.remove('grc-x-bible-unaccented')

    
train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_eng_fra_full_community.pickle", map_location=torch.device('cpu'))
#train_dataset, train_nodes_map = create_dataset(train_verses, verse_alignments_inter, small_editions)
features = train_dataset.features
train_nodes_map = train_dataset.nodes_map
#edge_index_intra_sent = train_dataset.edge_index_intra_sent
#test_edge_index_intra_sent = edge_index_intra_sent

# test_dataset, test_nodes_map = create_dataset(test_verses, verse_alignments_inter, small_editions)
test_dataset, test_nodes_map = train_dataset, train_nodes_map
test_verses = train_verses
print(train_dataset.x.shape)

# gutils.augment_features(test_dataset)
# x_edge, features_edge = gutils.create_edge_attribs(train_nodes_map, train_verses, small_editions, verse_alignments_inter, train_dataset.x.shape[0])
# with open("./dataset.pickle", 'wb') as of:
#     pickle.dump(train_dataset, of)


torch.Size([355584, 109])


ZeroDivisionError: division by zero

In [ ]:
# edges_intra_sent, edges_seq_sent = get_inter_sentence_connections(train_dataset.nodes_map)
# edge_index_seq_sent = torch.tensor(edges_seq_sent, dtype=torch.long)
# train_dataset.edge_index_seq_sent = edge_index_seq_sent
# torch.cuda.set_device(int(free_gpu1))
# edge_index_intra_sent = torch.tensor(edges_intra_sent, dtype=torch.long).to(dev)
# train_dataset.edge_index_intra_sent = edge_index_intra_sent
# test_edge_index_intra_sent = train_dataset.edge_index_intra_sent
# print(train_dataset.edge_index_intra_sent.shape)

In [ ]:
import pickle
import torch
## with open("./features_edge.pickle", 'wb') as of:
##     pickle.dump(features_edge, of)

## print('done first')
## with open("/mounts/work/ayyoob/models/gnn//x_edge.pickle", 'wb') as of:
##     pickle.dump(x_edge, of)

#with open("./features_edge.pickle", 'rb') as inf:
#    features_edge = pickle.load(inf)


## indices = [[],[]]
## values = []

## print('going to create sparse matrix representation')
## for i in range(len(ss)):
##     print(i)
##     for j in range(len(ss)):
##         if ss[i][j] != None and ss[i][j] != []:
##             indices[0].append(i)
##             indices[1].append(j)
##             values.append(ss[i][j])


## with open("./edge_attribs_sparse_indices.pickle", 'wb') as of:
##     pickle.dump(indices, of)

## with open("./edge_attribs_sparse_values.pickle", 'wb') as of:
##     pickle.dump(values, of)

## print('loading indices')
## with open("./edge_attribs_sparse_indices.pickle", 'rb') as inf:
##     indices = pickle.load(inf)

## print('loading values')
## with open("./edge_attribs_sparse_values.pickle", 'rb') as inf:
##     values = pickle.load(inf)

## print('creating sparse tensor')
## s = torch.sparse_coo_tensor(indices, values, (67800, 67800, len(ff)), dtype=torch.float16)
## print('saving sparse matrix')
## torch.save(s, "/mounts/work/ayyoob/models/gnn/edge_attribs_tensor16.pickle")

#print('loading sparse matrix')
#x_edge = torch.load("/mounts/work/ayyoob/models/gnn/edge_attribs_tensor.pickle")


#train_dataset.features_edge = features_edge



In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

#x_edge = x_edge.coalesce()
#torch.cuda.set_device(1)
#x_edge_vals = x_edge.values()
#indices_np = x_edge.indices().numpy()
#print(indices_np.shape)
#x_edge_np = csr_matrix((np.arange(indices_np.shape[1]), (indices_np[0, :], indices_np[1,:])), shape=(67800, 67800))

##x_edge_vals = x_edge_vals.cpu()
##maxes = torch.max(x_edge_vals,0)
##mins = torch.min(x_edge_vals,0)
##x_edge_vals_d = torch.div(x_edge_vals, maxes.values)

#print('creating targets')
#targets = torch.zeros(indices_np.shape[1], dtype=torch.int64)
#pos_indices = x_edge_np[train_dataset.edge_index.cpu().numpy()[0,:], train_dataset.edge_index.cpu().numpy()[1,:]]
#pos_indices = np.squeeze(np.asarray(pos_indices))
#targets[pos_indices] = 1
#print("done")

In [ ]:
 # run on delta, extract w2v features
#sys.path.insert(0, '../')
#import pickle
#from gensim.models import Word2Vec
#from app.document_retrieval import DocumentRetriever
#from my_utils import utils
#config_file = "/mounts/Users/student/ayyoob/Dokumente/code/pbc-ui-demo/config_pbc.ini"
#utils.setup(config_file)
#import torch
#import my_utils.alignment_features as feat_utils

#doc_retriever = DocumentRetriever()

#model_w2v = Word2Vec.load("word2vec_83langs_15epoch.model")
#train_dataset = torch.load("/mounts/work/ayyoob/models/gnn/dataset_en_fr_full.pickle")
#nodes_map = train_dataset.nodes_map

#x = [[] for i in range(train_dataset.x.shape[0])]
#for edition_f in nodes_map:
#    utils.LOG.info(f"processing edition {edition_f}")
#    for verse in nodes_map[edition_f]:         #toknom nodecount
#        line = doc_retriever.retrieve_document(f'{verse}@{edition_f}')
#        line = line.strip().split()

#        for tok in nodes_map[edition_f][verse]:
#            w_emb = model_w2v.wv[f'{edition_f[:3]}:{line[tok]}']
#            x[nodes_map[edition_f][verse][tok]].extend(w_emb)

#x = torch.tensor(x, dtype=torch.float)
#train_dataset.x = torch.cat((train_dataset.x, x), dim=1)
#train_dataset.features.append(feat_utils.ForwardFeature(50, 100, 'W2v'))

#print(x.shape, train_dataset.x.shape, len(train_dataset.features))

#torch.save(train_dataset, "/mounts/work/ayyoob/models/gnn/dataset_en_fr_full.pickle")

In [ ]:
## Add node embedding features
#importlib.reload(gutils)
#x_,features_ = gutils.get_embedding_node_features(train_dataset.nodes_map, train_verses, small_editions, verse_alignments_inter, x_edge_np, x_edge_vals.cpu().numpy())
#train_dataset.x = torch.cat((train_dataset.x,x_), dim=1)
#train_dataset.features.extend(features_)

In [14]:
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import random

class GNNDataset(Dataset):

    def __init__(self, dataset, verses, edit_files, alignments, group_size = 360):
        self.verses = list(verses)
        self.edit_files = list(edit_files)
        self.nodes_map = dataset.nodes_map

        self.items = self.calculate_size(self.nodes_map, self.verses, self.edit_files, group_size)
        self.alignments = alignments
        self.verse_info = {}
        self.calculate_verse_stats(verses, edit_files, alignments, dataset)
    
    def calculate_size(self, nodes_map, verses, edit_files, group_size):
        res = []
        item = []
        self.not_presented = []
        for verse in verses:
            if len(item) > 0:
                res.append(item)
                item = []
            if verse == verses[30]:
                self.length = len(res)
            for i,editf1 in enumerate(edit_files):
                if editf1 not in nodes_map:
                    self.not_presented.append(editf1)
                    continue
                if verse in nodes_map[editf1]:
                    for editf2 in edit_files[i+1:]:
                        if editf2 not in nodes_map:
                            self.not_presented.append(editf2)
                            continue
                        if verse in nodes_map[editf2]:
                            item.append((verse, editf1, editf2))
                            if len(item) >= group_size:
                                res.append(item)
                                item = []
        
        if len(item)>0:
            res.append(item)
        
        print(f"not presented: {set(self.not_presented)}")

        return res

    def calculate_verse_stats(self,verses, edition_files, alignments, dataset):
        min_edge = 0
        min_nodes = 0
        max_nodes = 0
        for verse in tqdm(verses):
            #utils.LOG.info(f"adding {verse}")
            edges_tmp = [[],[]]
            x_tmp = []
            features = []
            for i,editf1 in enumerate(edition_files):
                for j,editf2 in enumerate(edition_files[i+1:]):
                    aligns = autils.get_aligns(editf1, editf2, alignments[verse])
                    if aligns != None:
                        for align in aligns:
                            try:
                                n1,_ = gutils.node_nom(verse, editf1, align[0], None, dataset.nodes_map, x_tmp, edition_files, features)
                                n2,_ = gutils.node_nom(verse, editf2, align[1], None, dataset.nodes_map, x_tmp, edition_files, features)
                                edges_tmp[0].extend([n1, n2])

                                max_nodes = max(n1, n2, max_nodes)
                            except Exception as e:
                                print(editf1, editf2, verse)
                                raise(e)

            self.verse_info[verse] = {}

            self.verse_info[verse]['padding'] = min_nodes

            self.verse_info[verse]['x'] = dataset.x[min_nodes:max_nodes+1,:]
            
            self.verse_info[verse]['edge_index'] = dataset.edge_index[:, min_edge: min_edge + len(edges_tmp[0])] - min_nodes
            min_nodes = max_nodes + 1
            min_edge = min_edge + len(edges_tmp[0])

            
    
    def __len__(self):
        #return self.length
        return len(self.items)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        item = self.items[idx]

        res_pos = [[],[]]
        res_neg = [[],[]]
        nodes = set()
        for instance in item:
            verse, editf1, editf2 = instance
            aligns = autils.get_aligns(editf1, editf2, self.alignments[verse]) 
            if aligns != None:
                for align in aligns:
                    p1, p2 = align
                    n1 = self.nodes_map[editf1][verse][p1] - self.verse_info[verse]['padding']
                    n2 = self.nodes_map[editf2][verse][p2] - self.verse_info[verse]['padding']
                    res_pos[0].extend([n1,n2])
                    res_pos[1].extend([n2,n1])

                    n2_ = random.choice( list(self.nodes_map[editf2][verse].values()) ) - self.verse_info[verse]['padding']
                    n1_ = random.choice( list(self.nodes_map[editf1][verse].values()) ) - self.verse_info[verse]['padding']
                    
                    if n2_ != n2:
                        res_neg[0].extend([n1, n2_])
                        res_neg[1].extend([n2_, n1])
                    
                    if n1_ != n1:
                        res_neg[0].extend([n1_, n2])
                        res_neg[1].extend([n2, n1_])
                    
                    nodes.update([n1, n2, n1_, n2_])
                
        
        return {'pos':res_pos, 'neg':res_neg, 'nodes':nodes, 'verse':verse, 'editf1':editf1, 'editf2':editf2}

def collate_fun(input):
    res = {}
    #all_edits = {}
    for item in input:
        verse = item['verse'] 
        if verse not in res:
            res[verse] = {'pos': [[],[]], 'neg' : [[],[]],
                 'x':gnn_dataset.verse_info[verse]['x'], 'edge_index':gnn_dataset.verse_info[verse]['edge_index']
                 ,'intra_sent_edges':[[],[]]}
        
        res[verse]['pos'][0].extend(item['pos'][0])
        res[verse]['pos'][1].extend(item['pos'][1])

        res[verse]['neg'][0].extend(item['neg'][0])
        res[verse]['neg'][1].extend(item['neg'][1])

        #if verse not in all_edits:
        #    all_edits[verse] = []

        #if item['editf1'] not in all_edits[verse]:
        #    e = eval_utils.get_all_edges(verse, item['editf1'], train_dataset.nodes_map, gnn_dataset.verse_info)
        #    res[verse]['intra_sent_edges'][0].extend(e[0])
        #    res[verse]['intra_sent_edges'][1].extend(e[1])

        #if item['editf2'] not in all_edits[verse]:
        #    e = eval_utils.get_all_edges(verse, item['editf2'], train_dataset.nodes_map, gnn_dataset.verse_info)
        #    res[verse]['intra_sent_edges'][0].extend(e[0])
        #    res[verse]['intra_sent_edges'][1].extend(e[1])

    #nodes = list(nodes)
    #mapping = {node:pos for pos, node in enumerate(nodes)}
    ##indices = [[i for i in range(len(res_pos[0]))],[i for i in range(len(res_pos[1]))]]
    #indices = [[],[]]
    ##adjacency = torch.zeros((len(nodes), len(nodes)), dtype=torch.float)
    #for i in range(len(res_pos[0])):
    ##    adjacency[mapping[res_pos[0][i]], mapping[res_pos[1][i]]] = 1
    #    indices[0].append(mapping[res_pos[0][i]])
    #    indices[1].append(mapping[res_pos[1][i]])

    #adjacency = torch.sparse_coo_tensor(indices, [1 for i in range(len(res_pos[0]))], (len(nodes), len(nodes)))
    return res

gnn_dataset = GNNDataset(train_dataset, train_verses, small_editions, verse_alignments_inter)
len(gnn_dataset)

  1%|          | 3/250 [00:00<00:08, 28.42it/s]

not presented: set()


100%|██████████| 250/250 [00:17<00:00, 14.64it/s]


1180

In [15]:
from gnn_utils.eval_utils import *
train_dataset.train_mask = train_dataset.val_mask = train_dataset.test_mask = train_dataset.y = None
test_dataset.train_mask = test_dataset.val_mask = test_dataset.test_mask = test_dataset.y = None

# free_gpu = gpu_utils.wait_for_free_gpus()
torch.cuda.set_device(0)
print("device: ", dev)
# x_test, test_dataset.edge_index, test_dataset.neg_edge_index = test_dataset.x.to(dev), test_dataset.edge_index.to(dev), test_dataset.neg_edge_index.to(dev)
x = train_dataset.x
#edge_index_intra_sent, edge_index_seq_sent = train_dataset.edge_index_intra_sent.to(dev), train_dataset.edge_index_seq_sent.to(dev)
#edge_index_intra_sent, edge_index_seq_sent = torch.tensor([[],[]], dtype=torch.long).to(dev), train_dataset.edge_index_seq_sent.to(dev)
#test_edge_index_intra_sent = edge_index_intra_sent

train_pos_edge_index = train_dataset.edge_index
x_test, test_dataset.edge_index = x, train_pos_edge_index
features = train_dataset.features

device:  cuda


In [17]:
from tqdm import tqdm
features_edge = [] #TODO remove me
#features_edge = train_dataset.features_edge[:]
#x_edge_vals2 = x_edge_vals[:, :]
#features = train_dataset.features

data_loader = DataLoader(gnn_dataset, batch_size=1, collate_fn=collate_fun, shuffle=True)

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

drop_out = 0
pos_noise = 0.0
neg_noise = 0.0
n_head = 1


batch_size = 100
channels = 256

in_dim = sum(t.out_dim for t in features)
decoder_in_dim = n_head * channels * 2 + sum(t.out_dim for t in features_edge) 
print('edge features size: ', sum(t.out_dim for t in features_edge))
#discriminator = Discriminator(channels*n_head, channels * (n_head+1), channels*n_head)
#discriminator_optimizer = torch.optim.AdamW(discriminator.parameters(), lr=0.0007)
#encoder2 = Encoder2(channels, int(channels/2)).to(dev)
decoder = Decoder(decoder_in_dim, int(decoder_in_dim/2), features_edge, n_cluster=64)
model = pyg_nn.GAE(Encoder(in_dim, channels, features, n_head, edge_feature_dim=len(features_edge)), decoder).to(dev)
#model.encoder2 = encoder2
#model = pyg_nn.GAE(DeeperGCN(in_dim, len(features_edge), channels, 10, features), decoder=decoder).to(dev)
#model = pyg_nn.GAE(Encoder(in_dim, channels, features, n_head)).to(dev)

print("sending input to gpu")
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S-") + f"samett-{channels}chs-feat{train_dataset.num_node_features}-")

torch.set_printoptions(edgeitems=5)
print("model params - decoder params - conv1", sum(p.numel() for p in model.parameters()), sum(p.numel() for p in decoder.parameters()))

for epoch in range(1, 120):
    print(f"\n----------------epoch {epoch} ---------------")
    
    #if epoch % 1 == 0:
    #    train_neg_edge_index = gutils.get_negative_edges(train_verses, small_editions, train_dataset.nodes_map,  verse_alignments_inter).to(dev)
        #edge_index_seq_sent_neg = get_negative_edges_seq(train_dataset.nodes_map).to(dev)

    train(epoch)
    if epoch % 1 == 0:
        #alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses[:30], test_nodes_map,
        #    dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)
        alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses[:], test_nodes_map,
            dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)
        # auc, ap = test(edge_index_seq_sent, edge_index_seq_sent_neg, epoch)
        # print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    gc.collect()
    with torch.no_grad():
        torch.cuda.empty_cache()

  0%|          | 0/1180 [00:00<?, ?it/s]

edge features size:  0
sending input to gpu
model params - decoder params - conv1 733897 394241

----------------epoch 1 ---------------


100%|██████████| 1180/1180 [01:41<00:00, 11.68it/s]


train loss: 3043716.6624908447
cluster loss: 373.76904296875


  0%|          | 0/1180 [00:00<?, ?it/s]

...

train loss: 526976.4919986725
cluster loss: 69.06243133544922


  0%|          | 0/1180 [00:00<?, ?it/s]



intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.778, rec: 0.748, F1: 0.763, AER: 0.237
argmax prec: 0.951, rec: 0.657, F1: 0.777, AER: 0.22
resnorm prec: 0.805, rec: 0.738, F1: 0.77, AER: 0.229
itermax2-.9 prec: 0.844, rec: 0.724, F1: 0.779, AER: 0.218
itermax2-.95 prec: 0.843, rec: 0.724, F1: 0.779, AER: 0.219
itermax2-.8 prec: 0.844, rec: 0.724, F1: 0.779, AER: 0.218
my_gd prec: 0.927, rec: 0.701, F1: 0.798, AER: 0.199
my_gd_gdfa prec: 0.867, rec: 0.781, F1: 0.822, AER: 0.177
new1 prec: 0.913, rec: 0.682, F1: 0.781, AER: 0.216
new_mygd prec: 0.851, rec: 0.781, F1: 0.814, AER: 0.184
sum1 prec: 0.782, rec: 0.783, F1: 0.782, AER: 0.218

----------------epoch 119 ---------------


100%|██████████| 1180/1180 [01:25<00:00, 13.77it/s]


train loss: 522728.56561660767
cluster loss: 68.53446197509766


intersection prec: 0.971, rec: 0.521, F1: 0.678, AER: 0.319
gdfa prec: 0.856, rec: 0.71, F1: 0.776, AER: 0.221
my_gdfa prec: 0.777, rec: 0.75, F1: 0.763, AER: 0.236
argmax prec: 0.953, rec: 0.657, F1: 0.778, AER: 0.22
resnorm prec: 0.787, rec: 0.739, F1: 0.762, AER: 0.237
itermax2-.9 prec: 0.847, rec: 0.723, F1: 0.78, AER: 0.218
itermax2-.95 prec: 0.847, rec: 0.723, F1: 0.78, AER: 0.218
itermax2-.8 prec: 0.847, rec: 0.723, F1: 0.78, AER: 0.218
my_gd prec: 0.925, rec: 0.703, F1: 0.799, AER: 0.198
my_gd_gdfa prec: 0.866, rec: 0.783, F1: 0.822, AER: 0.175
new1 prec: 0.904, rec: 0.689, F1: 0.782, AER: 0.215
new_mygd prec: 0.844, rec: 0.786, F1: 0.814, AER: 0.185
sum1 prec: 0.782, rec: 0.786, F1: 0.784, AER: 0.216


In [ ]:
from gnn_utils import eval_utils
importlib.reload(eval_utils)
eval_utils.alignment_test(epoch, test_dataset.edge_index, editf1, editf2, test_verses[:], test_nodes_map,
                        dev, model, x_test, pros, surs, verse_alignments_inter, verse_alignments_gdfa, writer, gnn_dataset.verse_info)



intersection prec: 0.976, rec: 0.51, F1: 0.67, AER: 0.328
gdfa prec: 0.861, rec: 0.703, F1: 0.774, AER: 0.223
my_gdfa prec: 0.797, rec: 0.736, F1: 0.765, AER: 0.233
argmax prec: 0.957, rec: 0.638, F1: 0.766, AER: 0.231
resnorm prec: 0.764, rec: 0.755, F1: 0.759, AER: 0.24
itermax2-.9 prec: 0.854, rec: 0.71, F1: 0.775, AER: 0.221
itermax2-.95 prec: 0.854, rec: 0.711, F1: 0.776, AER: 0.221
itermax2-.8 prec: 0.855, rec: 0.709, F1: 0.775, AER: 0.222
my_gd prec: 0.924, rec: 0.704, F1: 0.799, AER: 0.198
my_gd_gdfa prec: 0.855, rec: 0.786, F1: 0.819, AER: 0.18
new1 prec: 0.916, rec: 0.676, F1: 0.778, AER: 0.218
new_mygd prec: 0.84, rec: 0.782, F1: 0.81, AER: 0.188
sum1 prec: 0.802, rec: 0.785, F1: 0.793, AER: 0.207


0.799

In [ ]:
global model, decoder
1/0

decoder = None
model = None

gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
features = train_dataset.features[:]
#features_edge = train_dataset.features_edge[:]
from pprint import pprint
print('indim',in_dim)
#features[-1].out_dim = 50
for i in features:
    if i.type==3:
        i.out_dim=4
    print(vars(i))

sum(p.out_dim for p in features)



indim 130
{'type': 1, 'out_dim': 20, 'global_normalize': False, 'name': 'edit_f', 'Active': True, 'n_classes': 81}
{'type': 1, 'out_dim': 32, 'global_normalize': False, 'name': 'position', 'Active': True, 'n_classes': 100}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'degree_centrality', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'katz_centrality_numpy', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'closeness_centrality', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'betweenness_centrality', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'load_centrality', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'subgraph_centrality', 'Active': True}
{'type': 3, 'out_dim': 4, 'global_normalize': False, 'name': 'harmonic_centrality', 'Active': True}
{'type': 4, 'out_dim': 50, 'global_normalize': False, 'name': 'W2v', 'Active'

130

In [ ]:
# count number of deleted edges by each community detection method
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# tmp_verses = [all_verses[2]]
# tmp_editions = small_editions[:10]
# tmp_dataset, tmp_nodes_map = create_dataset(tmp_verses, verse_alignments_inter, tmp_editions)

# tmp_g = pyg_utils.convert.to_networkx(tmp_dataset, to_undirected=True)
# def count_deleted_edges(tmp_dataset, c):
#     deleted_edges = 0
#     for i in range(0, len(tmp_dataset.edge_index[0]), 2):
#         for comp in c:
#             if tmp_dataset.edge_index[0][i].item() in comp and tmp_dataset.edge_index[1][i].item() not in comp:
#                 deleted_edges += 1
    
#     return deleted_edges

# print("eng token count: ", tmp_nodes_map['eng-x-bible-mixed'][tmp_verses[0]])
# print("original connected components",nx.number_connected_components(tmp_g))

# c = list(greedy_modularity_communities(tmp_g))
# print("new connected_components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(asyn_lpa_communities(tmp_g))
# print("asyn_lpa_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# c = list(label_propagation_communities(tmp_g))
# print("label_propagation_communities number of components", len(c))
# print("deleted edges: ", count_deleted_edges(tmp_dataset, c))

# cents = nx.edge_betweenness_centrality(tmp_g)
# vals = sorted(list(cents.values()))
# print(vals[0], vals[10], vals[100], vals[1000], vals[2000], vals[3000], vals[10000])
# print(vals[-1], vals[-10], vals[-100], vals[-1000], vals[-2000], vals[-3000], vals[-10000])



In [ ]:
# measure different community detection algorithms
# from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

# def remove_bad_community_edges(nodes_map, verses, edition_files, alignments):
#     edges_tmp = [[],[]]
#     res_edges = [[],[]]
#     for verse in verses:
#         utils.LOG.info(f"extracting edge features for {verse}")
#         for i,editf1 in enumerate(edition_files):
#             for j,editf2 in enumerate(edition_files[i+1:]):
#                 aligns = autils.get_aligns(editf1, editf2, alignments[verse])
#                 if aligns != None:
#                     for align in aligns:
#                         n1, node_count = node_nom(verse, editf1, align[0], 0, nodes_map, None, None)
#                         n2, node_count = node_nom(verse, editf2, align[1], 0, nodes_map, None, None)
#                         edges_tmp[0].extend([n1, n2])
#                         edges_tmp[1].extend([n2, n1])

#         gnx = convert_to_netx(edges_tmp)
#         print('detecting communities')
#         coms = greedy_modularity_communities(gnx)

#         print('finding good edges')
#         for i in range(0, len(edges_tmp[0]), 2):
#             for c in coms:
#                 if edges_tmp[0][i] in c and edges_tmp[0][i+1] in c:
#                     res_edges[0].extend([edges_tmp[0][i], edges_tmp[0][i+1]])
#                     res_edges[1].extend([edges_tmp[0][i+1], edges_tmp[0][i]])
#         edges_tmp = [[],[]]
#     print('to keep edges:', len(res_edges[0]))
#     return torch.tensor(res_edges, dtype=torch.long)

# # old_edge_index = train_dataset.edge_index
# # new_edge_index = remove_bad_community_edges(train_dataset.nodes_map, train_verses, small_editions, verse_alignments_inter)
# # train_dataset.edge_index = new_edge_index

# # with open("./dataset_greedy_modularity_communities.pickle", 'rb') as inf:
# #     train_dataset = pickle.load(inf)

# test_dataset = train_dataset

# print('orig edge count', old_edge_index.shape)
# print('new edge count', train_dataset.edge_index.shape)
# print("done")

In [ ]:
nodes_map = train_dataset.nodes_map
bad_edition_files = []
for edit in nodes_map:
    bad_count = 0
    for verse in nodes_map[edit]:
        if len(nodes_map[edit][verse].keys()) < 6:
            bad_count += 1
        if bad_count > 1:
            bad_edition_files.append(edit)
            break
print(bad_edition_files)

['mya-x-bible-1835']


In [ ]:
all_japanese_nodes = set()
nodes_map = train_dataset.nodes_map

for verse in nodes_map['jpn-x-bible-newworld']:
    for item in nodes_map['jpn-x-bible-newworld'][verse].items():
        all_japanese_nodes.add(item[1])

print(" all japansese nodes: ", len(all_japanese_nodes))
edge_index = train_dataset.edge_index.to('cpu')
remaining_edges_index = []
for i in tqdm(range(0, edge_index.shape[1], 2)):
    if edge_index[0, i].item() not in all_japanese_nodes and edge_index[0, i+1].item() not in all_japanese_nodes:
        remaining_edges_index.extend([i, i+1])

print('original total edges count', edge_index.shape)
print('remaining edge count', len(remaining_edges_index))
train_dataset.edge_index = edge_index[:, remaining_edges_index]
train_dataset.edge_index.shape


KeyError: 'jpn-x-bible-newworld'

In [ ]:
from networkx.algorithms.community import greedy_modularity_communities, asyn_lpa_communities, label_propagation_communities, asyn_fluidc

def get_community_edges(c, verse):
    res = []
    for n1 in all_nodes_map[editf1][verse].items():
        for n2 in all_nodes_map[editf2][verse].items():
            for com in c:
                if n1[1] in com and n2[1] in com:
                    res.append((n1[0], n2[0]))
    
    return res

def intersect(e1, e2):
    res = set()
    for item in e1:
        if item in e2:
            res.add(item)
    
    return res


measures = {}
measures['intersection']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['gdfa']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c1_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c2_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_all']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
measures['c3_inter']= {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
numbers = [100, 300, 600, 1000, 1300, 1600, 2000, 3000, 4000, 5000]
for i in numbers:
    measures[i] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
    measures[i+1] = {"p_hit_count": 0, "s_hit_count": 0, "total_hit_count": 0, "gold_s_hit_count": 0, "prec": 0, "rec": 0, "f1": 0, "aer": 0}
coms = {}
for verse in test_verses:
    inter_edges = autils.get_aligns(editf1, editf2, verse_alignments_inter[verse])

    all_dataset, all_nodes_map = create_dataset([verse], verse_alignments_inter, small_editions)
    print("converting")
    g = pyg_utils.convert.to_networkx(all_dataset, to_undirected=True)
    print("detecting community 1")
    c1 = list(greedy_modularity_communities(g))
    print("detecting community 2")
    c2 = list(asyn_lpa_communities(g))
    print("detecting community 3")
    c3 = list(label_propagation_communities(g))

    print("detecting community 4")
    edge_betweenness = [y[0] for y in sorted(nx.edge_betweenness_centrality(g).items(), key=lambda x: x[1], reverse=True)]
    
    print('orig communities', nx.number_connected_components(g))
    prev_i = 0
    for i in numbers:
        for j in range(prev_i, i):
            g.remove_edge(*edge_betweenness[j])
        prev_i = i
        com = list(nx.connected_components(g))
        edges = get_community_edges(com, verse)
        autils.calc_and_update_alignment_score(edges, pros[verse], surs[verse], measures[i])
        autils.calc_and_update_alignment_score(intersect(edges, inter_edges), pros[verse], surs[verse], measures[i+1]) 
        print(f'communities {i}', nx.number_connected_components(g))

    c1_edges = get_community_edges(c1, verse)
    c2_edges = get_community_edges(c2, verse)
    c3_edges = get_community_edges(c3, verse)
    print('c1 communities', len(c1))
    print('c2 communities', len(c2))
    print('c3 communities', len(c3))



    autils.calc_and_update_alignment_score(inter_edges, pros[verse], surs[verse], measures['intersection'])
    autils.calc_and_update_alignment_score(autils.get_aligns(editf1, editf2, verse_alignments_gdfa[verse]), pros[verse], surs[verse], measures['gdfa'])

    autils.calc_and_update_alignment_score(c1_edges, pros[verse], surs[verse], measures['c1_all'])
    autils.calc_and_update_alignment_score(c2_edges, pros[verse], surs[verse], measures['c2_all'])
    autils.calc_and_update_alignment_score(c3_edges, pros[verse], surs[verse], measures['c3_all'])

    autils.calc_and_update_alignment_score(intersect(c1_edges, inter_edges), pros[verse], surs[verse], measures['c1_inter'])
    autils.calc_and_update_alignment_score(intersect(c2_edges, inter_edges), pros[verse], surs[verse], measures['c2_inter'])
    autils.calc_and_update_alignment_score(intersect(c3_edges, inter_edges), pros[verse], surs[verse], measures['c3_inter'])

    for item in measures:
        print(item, measures[item])
    
    
